# Подзадача 1
## Сбор (отбор) и предварительная обработка данных (предобработка данных).

__Исходный материал:__ видео, снятое с камеры летящего БПЛА (https://cloud.sketcher-services.ru/s/tBHnPdoBF22sfdt). 

__Задача:__ на основе материалов предоставленного исходного материала видео отобрать и подготовить релевантные данные для обучения модели. 

In [1]:
pip install natsort

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import os
import natsort as ns

## 1.1 Отбор кадров из видео с помощью автоматической функции

In [2]:
def video2frames(src, out, sample, xr, yr):
    
    if not os.path.exists(out):
        os.mkdir(out)
    
    cap = cv2.VideoCapture(src)
    
    if not cap.isOpened(): 
        print('Ошибка чтения видео-файла')
    
    i, s = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            if i % sample == 0:
                frame = cv2.resize(frame, (xr, yr), cv2.INTER_NEAREST)
                cv2.imwrite(os.path.join(out, src.split('.')[0] + '_' + str(i) + '.jpg'), frame)
                s += 1
            i += 1
        else:
            break
    cap.release()
        
    return f'Задача завершена. Сохранено кадров: {s}.'

In [7]:
# загрузка видео и определение директорий
src = '2.mp4'
out = os.path.join(os.path.dirname(src), 'frames')
cap = cv2.VideoCapture(src)

# задаем размеры видео
new_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
new_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sample = cap.get(cv2.CAP_PROP_FRAME_COUNT) // 50

In [8]:
%%time

video2frames(src, out, sample, new_x, new_y)

CPU times: user 57.8 s, sys: 2.57 s, total: 1min
Wall time: 51.5 s


'Задача завершена. Сохранено кадров: 56.'

## 1.2. Создание выборки вырезок из отобранных кадров с помощью автоматической функции (также предоставлена преподавателями)

In [9]:
def click_sampler(name, size, out):
    
    if not os.path.exists(out):
        os.mkdir(out)
        
    if size[0] <= 0 or size[1] <= 0:
        raise ValueError ('Размер вырезки должен быть больше 0')
  
    img = cv2.imread(name)
    
    if size[0] > img.shape[0] or size[1] > img.shape[1]:
        print('Внимание: размер вырезки больше размера изображения')
        pass
    
    img_c = img.copy()

    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.resizeWindow(name, img.shape[1], img.shape[0])

    coords = []

    def mouse_click(event, x, y, flags, param):
        
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(img, (x, y), 10, (0, 0, 255), -1)
            x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
            xw, yh = x + int(size[0]/2), y + int(size[1]/2)
            cv2.rectangle(img, (x0, y0), (xw, yh), (0, 0, 255), 3)
            cv2.imshow(name, img)
            coords.append((x, y))

        return coords

    cv2.setMouseCallback(name, mouse_click)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    t = 0
    nm, ext = os.path.basename(name.split('.')[0]), os.path.basename(name.split('.')[1])
    
    for coord in coords:
        x, y = coord[0], coord[1]
        x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
        xw, yh = x + int(size[0]/2), y + int(size[1]/2)
        
        if x0 < 0:
            xw = xw - x0
            x0 = 0
        if y0 < 0:
            yh = yh - y0
            y0 = 0
            
        title = img_c[y0:yh, x0:xw]
        print(f"Создан фрейм {os.path.join(out, nm + '_' + str(x)+ 'x' + str(y) + '.' + ext)}")
        cv2.imwrite(os.path.join(out, nm + '_' + str(x) + 'x' + str(y) + '.' + ext), title)
                    
        t += 1
        
    return f'Создано вырезок: {t}'

In [1]:
# Применяем скрипт click_sampler итеративно для всех изображений в папке frame
pth = 'frames'
out = '640x640'

out_pth = os.path.join(pth, out)

if not os.path.exists(out_pth):
    os.mkdir(out_pth)

for file in ns.natsorted(os.listdir(pth)):
    print(os.path.join(out_pth, os.path.basename(file)))
    click_sampler(os.path.join(pth, os.path.basename(file)), (640,640), out_pth)

NameError: name 'os' is not defined

## Итог работы:
* Набор вырезок 640х640 пикс из кадров (199 объектов), релевантных целевым объектам (автомобили и здания).
* Сформированный набор: в папке ./frames/640x640